# Prototype Curves – Week 1

**Goals**
1. Load `docs/literature_matrix.csv`.
2. Plot F1 vs. annotator minutes for *Dragut et al., 2019*.
3. Plot accuracy vs. dollars for *Kang et al., 2023*.
4. Illustrate diminishing returns & marginal-gain stop point.

_Code cells will be added once I’m on a machine with Jupyter._
